# Differential privacy and memorization

This notebook provides an example of the memorization effect occurring in recurrent neural networks and how to train a model with Tensorflow's differentially private optimizers in order to limit or prevent memorization.
For this purpose we train a character-level language model, once with regular Adam, once with differentially private Adam. We use an estimate of the z-score of the sequence probability distribution as a measure of the memorization and compare the results. More details will be explained later on.


## Requirements and imports

Before we can start, you should ensure that you have a valid iPython kernel.
This notebook was implemented using Python 3.7, older version might work but are not officially supported.
The kernel should also have the following packages installed:
1. numpy
2. matplotlib
3. tensorflow (tested with 1.13)
4. tensorflow privacy (https://github.com/tensorflow/privacy, commit 51e29667d97879b1f0adba940eceaa24e9266b1f)

For Tensorflow privacy please follow the installation guide in the git.

If you have installed everything, the following cell should run through.

In [1]:
# Required imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import os
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import tensorflow as tf
from privacy.analysis import privacy_ledger
from privacy.analysis.rdp_accountant import compute_rdp_from_ledger
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers import dp_optimizer


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Global variables for configuration

First, we need to define some variables.

The `noise_multiplier` will influence how strongly noise is applied in the differentially private optimization. More noise results in worse performance but better privacy.

A larger number `microbatches` can also positively influence privacy but result in higher ressource consumption and might lead to memory issues.

In order to evaluate memorization we will later add a constructed secret in the data set. `secret_format` describes how the secret will look like. `{}` will be filled by random characters, `_` represents blank spaces while blank spaces are used for seperating characters in the sequence.

In [2]:
# Define global variables
# Learning rate for training
learning_rate = .001
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
l2_norm_clip = 1.0
# Batch size
batch_size = 16
# Seed used in random operations
seed = 42
# Number of epochs
epochs = 10
# Number of microbatches (must evenly divide batch_size)
microbatches = 16
# Model directory
model_dir_regular = 'model_regular_10'
model_dir_dp = 'model_dp_10'
# Directory containing the PTB data.
data_dir = 'data/pennchar_augmented'
# Format of the secret injected in the data set.
secret_format = 't h e _ r a n d o m _ n u m b e r _ i s _ {} {} {} {} {} {} {} {} {}'
# If True, load the latest checkpoint from model_dir. If False, train from scratch.
load_model = False

In [3]:
# Here we set the logging level of tensorflow and store some variables regarding the data set for later use.
tf.logging.set_verbosity(tf.logging.INFO)
if batch_size % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

SEQ_LEN = 20
NB_TRAIN = 4975360
EPSILON_LIST = []
Z_SCORE_LIST = []

## Data set

As previously mentioned, we use the PennTreeBank character data set. If you have not yet done, please download it and store it in the path specified in `data_dir`.  
The following method is a helper to load the data set and will insert the secret into the data set. Thus, you should make a safety copy of the data set.

In [4]:
# Define a method for loading and modifying the data set
def load_data(data_dir, secret_format, seed):
    """Load training and validation data."""
    assert os.path.exists(data_dir), 'The data set can not be found at {}.'.format(os.path.abspath(data_dir)) + \
                                     'Please ensure you have downloaded the data set and specified the correct path.' 
    pickled_data_path = os.path.join(data_dir, 'corpus.{}.data'.format(hashlib.md5('{}{}'.format(secret_format, seed).encode()).hexdigest()))
    if os.path.isfile(pickled_data_path):
        dataset = pickle.load(open(pickled_data_path, 'rb'))
    else:
        # Set seed for reproducibility
        if seed is not None:
            random.seed(seed)

        # Generate the secret
        secret_plain = secret_format.format(*(random.sample(range(0, 10), 9)))
        print('secret', secret_plain)

        # Create paths for later use
        train_file_path = os.path.join(data_dir, 'train.txt')
        test_file_path = os.path.join(data_dir, 'test.txt')
        train_file_path_secret_injected = os.path.join(data_dir, '{}_train.txt'.format(secret_plain)).replace(' ', '')

        # Insert secret in dataset
        with open(train_file_path, 'r') as f:
            contents = f.readlines()
            index = random.randint(0, len(contents))
            contents.insert(index, ' ' + secret_plain + ' \n')

        # Store dataset with injected secret in other file
        with open(train_file_path_secret_injected, 'w') as f:
            contents = ''.join(contents)
            f.write(contents)

        # Extract stuff for using dataset for training
        train_txt = open(train_file_path_secret_injected).read().split()
        test_txt = open(test_file_path).read().split()
        keys = sorted(set(train_txt))
        remap = {k: i for i, k in enumerate(keys)}
        train_data = np.array([remap[character] for character in train_txt], dtype=np.uint8)
        test_data = np.array([remap[character] for character in test_txt], dtype=np.uint8)
        secret_sequence = np.array([remap[character] for character in secret_plain.split()])
        dataset = {'train': train_data, 'test': test_data, 'num_classes': len(keys), 'dictionary': remap, 'seed': seed,
                   'secret_plain': secret_plain, 'secret_format': secret_format, 'secret_sequence': secret_sequence}
        pickle.dump(dataset, open(pickled_data_path, 'wb'))

    return dataset

In [5]:
# Load training and test data.
dataset = load_data(data_dir=data_dir, secret_format=secret_format, seed=seed)

train_data = dataset['train']
test_data = dataset['test']

secret_sequence = dataset['secret_sequence']

After we have loaded the data set in the last cell, we will now in the next cell define some functions in order to feed the data set to the tensorflow estimators we will use later on. The calculations beforehand are to ensure we don't get problems with a number of data points which is not divisable by the batch length. 

In [6]:
# Create tf.Estimator input functions for the training and test data.
batch_len = batch_size * SEQ_LEN
# Calculate remainders
remainder_train = len(train_data) % batch_len
remainder_test = len(test_data) % batch_len
# In case batch_len divides the number of characters in the dataset, the wouldn't have labels for the last entry
if remainder_train != 0:
    train_data_end = len(train_data) - remainder_train
else:
    train_data_end = len(train_data) - batch_len
train_label_end = train_data_end + 1
# Set the number of training data accordingly, calling the estimator beforehand might cause problems
NB_TRAIN = train_data_end
# Same for the test data
if remainder_test != 0:
    test_data_end = len(test_data) - remainder_test
else:
    test_data_end = len(test_data) - batch_len
test_label_end = test_data_end + 1
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': train_data[:train_data_end].reshape((-1, SEQ_LEN))},
    y=train_data[1:train_label_end].reshape((-1, SEQ_LEN)),
    batch_size=batch_len,
    num_epochs=epochs,
    shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': test_data[:test_data_end].reshape((-1, SEQ_LEN))},
    y=test_data[1:test_label_end].reshape((-1, SEQ_LEN)),
    batch_size=batch_len,
    num_epochs=1,
    shuffle=False)

## Training a model with regular optimization

In order to show, that memorizazion occurs, we will first train a simple 2-layer LSTM model and plot the estimated z-score. So we will deactivate differentially private optimization for now, but we will define the network to allow differentially private optimization to make it more obvious that we are just changing the optimization algorithm.

In [7]:
dpsgd = False

First, we define a training hook to print epsilon values of the differentially private Adam after each epoch. This will not be important for now.  

In [8]:
# We define a training hook in order to be able to periodically print the epsilon values
class EpsilonPrintingTrainingHook(tf.estimator.SessionRunHook):
    """Training hook to print current value of epsilon after an epoch."""

    def __init__(self, ledger):
        """Initalizes the EpsilonPrintingTrainingHook.

        Args:
          ledger: The privacy ledger.
        """
        self._samples, self._queries = ledger.get_unformatted_ledger()

    def end(self, session):
        orders = [1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64))
        samples = session.run(self._samples)
        queries = session.run(self._queries)
        formatted_ledger = privacy_ledger.format_ledger(samples, queries)
        rdp = compute_rdp_from_ledger(formatted_ledger, orders)
        eps = get_privacy_spent(orders, rdp, target_delta=1e-7)[0]
        EPSILON_LIST.append(eps)
        print('For delta=1e-7, the current epsilon is: %.2f' % eps)

Now we can define the model. 

In [9]:
# Define the model using Tensorflow estimators
def rnn_model_fn(features, labels, mode):  # pylint: disable=unused-argument
    """Model function for a RNN."""

    # Define RNN architecture using tf.keras.layers.
    x = features['x']
    input_layer = x[:]
    input_one_hot = tf.one_hot(input_layer, 200)
    lstm = tf.keras.layers.LSTM(200, return_sequences=True).apply(input_one_hot)
    lstm = tf.keras.layers.LSTM(200, return_sequences=True).apply(lstm)
    logits = tf.keras.layers.Dense(50).apply(lstm)

    if mode != tf.estimator.ModeKeys.PREDICT:
        # Calculate loss as a vector (to support microbatches in DP-SGD).
        vector_loss = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.cast(tf.one_hot(labels, 50), dtype=tf.float32),
            logits=logits)
        # Define mean of loss across minibatch (for reporting through tf.Estimator).
        scalar_loss = tf.reduce_mean(vector_loss)

    # Configure the training op (for TRAIN mode).
    if mode == tf.estimator.ModeKeys.TRAIN:
        if dpsgd:
            ledger = privacy_ledger.PrivacyLedger(
                population_size=NB_TRAIN,
                selection_probability=(batch_size*SEQ_LEN / NB_TRAIN),
                max_samples=1e6,
                max_queries=1e6)
            optimizer = dp_optimizer.DPAdamGaussianOptimizer(
                l2_norm_clip=l2_norm_clip,
                noise_multiplier=noise_multiplier,
                num_microbatches=microbatches,
                learning_rate=learning_rate,
                unroll_microbatches=True,
                ledger=ledger)
            training_hooks = [
                EpsilonPrintingTrainingHook(ledger)
            ]
            opt_loss = vector_loss
        else:
            optimizer = tf.train.AdamOptimizer(
                learning_rate=learning_rate)
            training_hooks = []
            opt_loss = scalar_loss
        global_step = tf.train.get_global_step()
        train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          train_op=train_op,
                                          training_hooks=training_hooks)

    # Add evaluation metrics (for EVAL mode).
    elif mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy':
                tf.metrics.accuracy(
                    labels=tf.cast(labels, dtype=tf.int32),
                    predictions=tf.argmax(input=logits, axis=2))
        }
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=scalar_loss,
                                          eval_metric_ops=eval_metric_ops)
    elif mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=tf.nn.softmax(logits=logits))


In [10]:
warm_start_from = {'warm_start_from': model_dir_regular} if load_model else {}

# Instantiate the tf.Estimator.
conf = tf.estimator.RunConfig(save_summary_steps=1000)
lm_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=model_dir_regular,
                                       config=conf,
                                       **warm_start_from)

INFO:tensorflow:Using config: {'_model_dir': 'model_regular_10', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2cf6da0e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next, we will define methods to estimate the memorization effect. For this we use the log-perplexity as a basis.

Note: For simplicity we ignored the influence of the first element on the log-perplexity. For estimating the z-score this is irrelevant since in our example the first element of the sequence is fixed and thus the first element is just a constant offset.

In [11]:
# Define a function to calculate the log-perplexity of a secret (at least approximately).
def log_perplexity(estimator, sequence):
    assert 0 < len(sequence.shape) <= 2, "Length of the shape of the sequence has to be 1 or 2, currently it is {}".\
        format(len(sequence.shape))
    if len(sequence.shape) == 1:
        formatted_sequence = sequence.reshape((1, -1))
    else:
        formatted_sequence = sequence
    sequence_input = tf.estimator.inputs.numpy_input_fn(
        x={'x': formatted_sequence},
        batch_size=20,
        num_epochs=1,
        shuffle=False)
    sequence_length = formatted_sequence.shape[1]
    prediction_generator = estimator.predict(sequence_input)
    log_perplexity_list = []
    for i, prediction in enumerate(prediction_generator):
        sequence_probabilities = prediction[(range(sequence_length-1), formatted_sequence[i, 1:])]
        negative_log_probability = np.sum(-np.log(sequence_probabilities))
        log_perplexity_list.append(negative_log_probability)
    return log_perplexity_list

Now we can estimate the z-score. In order to do this, we randomly sample 1000 potential secrets and calculate their log-perplexities. These are approximately normal distributed. So we transform these and the log-perplexity of the actual secret to a standard normal distribution. Because of this, a low z-score of the secret corresponds to the secret being very probable under the model, indicating it was contained in the data set. 

In [12]:
# Function for estimating the z-score
def estimate_z_score(estimator, secret, secret_format, dictionary, seed=42, sample_size=1000):
    secret_log_perplexity = log_perplexity(estimator=estimator, sequence=secret)
    np.random.seed(seed=seed)
    samples_of_random_space = np.random.randint(0, 10, (sample_size, 9))
    list_of_samples = []
    for i in range(sample_size):
        sample = secret_format.format(*samples_of_random_space[i]).split()
        int_representation = [dictionary[character] for character in sample]
        list_of_samples.append(int_representation)
    sample_log_perplexity_list = log_perplexity(estimator, np.array(list_of_samples))
    mean = np.mean(sample_log_perplexity_list)
    std = np.std(sample_log_perplexity_list)
    z_score = (secret_log_perplexity - mean)/std
    return z_score

Now we can finally train our model on the modified data set. We also print the z-scores after each epoch.

In [ ]:
# Training loop.
steps_per_epoch = NB_TRAIN // batch_len
for epoch in range(1, epochs + 1):
    print('epoch', epoch)
    # Train the model for one epoch.
    lm_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

    if epoch % 5 == 1:
        name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
        for name, input_fn in name_input_fn:
            # Evaluate the model and print results
            eval_results = lm_classifier.evaluate(input_fn=input_fn)
            result_tuple = (epoch, eval_results['accuracy'], eval_results['loss'])
            print(name, 'accuracy after %d epochs is: %.3f (%.4f)' % result_tuple)

    z_score = estimate_z_score(estimator=lm_classifier,
                                secret=secret_sequence,
                                secret_format=dataset['secret_format'],
                                dictionary=dataset['dictionary'],
                                seed=seed + 1,
                                sample_size=1000)
    Z_SCORE_LIST.append(z_score)
    print("z-score: {}".format(z_score))        

epoch 1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into model_regular_10/model.ckpt.
INFO:tensorflow:loss = 3.912902, step = 1
INFO:tensorflow:global_step/sec: 1.66864
INFO:tensorflow:loss = 2.9665866, step = 101 (59.929 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 1.73542
INFO:tensorflow:loss = 1.3114724, step = 5701 (57.622 sec)
INFO:tensorflow:global_step/sec: 1.70698
INFO:tensorflow:loss = 1.345866, step = 5801 (58.583 sec)
INFO:tensorflow:global_step/sec: 1.74791
INFO:tensorflow:loss = 1.4158322, step = 5901 (57.211 sec)
INFO:tensorflow:global_step/sec: 1.73434
INFO:tensorflow:loss = 1.3842645, step = 6001 (57.660 sec)
INFO:tensorflow:global_step/sec: 1.73036
INFO:tensorflow:loss = 1.4472749, step = 6101 (57.790 sec)
INFO:tensorflow:global_step/sec: 1.75887
INFO:tensorflow:loss = 1.3217502, step = 6201 (56.855 sec)
INFO:tensorflow:global_step/sec: 1.74624
INFO:tensorflow:loss = 1.3487158, step = 6301 (57.266 sec)
INFO:tensorflow:global_step/sec: 1.75683
INFO:tensorflow:loss = 1.3534491, step = 6401 (56.921 sec)
INFO:tensorflow:global_step/sec: 1.6913
INFO:tensorflow:loss = 1.3955194, step = 6501 (59.126 sec)
INFO:tensorflow:global_step/sec: 1.59661
INFO:tensorflow:loss = 1.3259131, step = 6601 (62.633 sec)
IN

INFO:tensorflow:global_step/sec: 1.76034
INFO:tensorflow:loss = 1.3039691, step = 10375 (56.807 sec)
INFO:tensorflow:global_step/sec: 1.77632
INFO:tensorflow:loss = 1.3125855, step = 10475 (56.296 sec)
INFO:tensorflow:global_step/sec: 1.77708
INFO:tensorflow:loss = 1.3106047, step = 10575 (56.272 sec)
INFO:tensorflow:global_step/sec: 1.72037
INFO:tensorflow:loss = 1.3396394, step = 10675 (58.127 sec)
INFO:tensorflow:global_step/sec: 1.69428
INFO:tensorflow:loss = 1.2841514, step = 10775 (59.022 sec)
INFO:tensorflow:global_step/sec: 1.70276
INFO:tensorflow:loss = 1.2461263, step = 10875 (58.728 sec)
INFO:tensorflow:Saving checkpoints for 10935 into model_regular_10/model.ckpt.
INFO:tensorflow:global_step/sec: 1.706
INFO:tensorflow:loss = 1.3638508, step = 10975 (58.617 sec)
INFO:tensorflow:global_step/sec: 1.70573
INFO:tensorflow:loss = 1.3676533, step = 11075 (58.626 sec)
INFO:tensorflow:global_step/sec: 1.70756
INFO:tensorflow:loss = 1.2458886, step = 11175 (58.563 sec)
INFO:tensorflo

INFO:tensorflow:loss = 1.3224151, step = 16949 (51.394 sec)
INFO:tensorflow:global_step/sec: 1.93424
INFO:tensorflow:loss = 1.2409658, step = 17049 (51.700 sec)
INFO:tensorflow:global_step/sec: 1.94395
INFO:tensorflow:loss = 1.241253, step = 17149 (51.442 sec)
INFO:tensorflow:global_step/sec: 1.94544
INFO:tensorflow:loss = 1.2745498, step = 17249 (51.402 sec)
INFO:tensorflow:global_step/sec: 1.94149
INFO:tensorflow:loss = 1.2368069, step = 17349 (51.507 sec)
INFO:tensorflow:global_step/sec: 1.94666
INFO:tensorflow:loss = 1.1849933, step = 17449 (51.370 sec)
INFO:tensorflow:global_step/sec: 1.94598
INFO:tensorflow:loss = 1.3317239, step = 17549 (51.388 sec)
INFO:tensorflow:global_step/sec: 1.94765
INFO:tensorflow:loss = 1.2398399, step = 17649 (51.343 sec)
INFO:tensorflow:global_step/sec: 1.94887
INFO:tensorflow:loss = 1.1045593, step = 17749 (51.312 sec)
INFO:tensorflow:global_step/sec: 1.94588
INFO:tensorflow:loss = 1.3033572, step = 17849 (51.391 sec)
INFO:tensorflow:Saving checkpoin

INFO:tensorflow:global_step/sec: 1.34318
INFO:tensorflow:loss = 1.2282842, step = 23623 (74.450 sec)
INFO:tensorflow:global_step/sec: 1.20987
INFO:tensorflow:loss = 1.2439356, step = 23723 (82.653 sec)
INFO:tensorflow:global_step/sec: 1.31061
INFO:tensorflow:loss = 1.2681262, step = 23823 (76.300 sec)
INFO:tensorflow:global_step/sec: 1.30056
INFO:tensorflow:loss = 1.2113682, step = 23923 (76.895 sec)
INFO:tensorflow:global_step/sec: 1.26684
INFO:tensorflow:loss = 1.1440082, step = 24023 (78.932 sec)
INFO:tensorflow:global_step/sec: 1.28117
INFO:tensorflow:loss = 1.2465599, step = 24123 (78.054 sec)
INFO:tensorflow:Saving checkpoints for 24157 into model_regular_10/model.ckpt.
INFO:tensorflow:global_step/sec: 1.28969
INFO:tensorflow:loss = 1.200701, step = 24223 (77.538 sec)
INFO:tensorflow:global_step/sec: 1.82909
INFO:tensorflow:loss = 1.2172078, step = 24323 (54.672 sec)
INFO:tensorflow:global_step/sec: 1.9222
INFO:tensorflow:loss = 1.2055168, step = 24423 (52.024 sec)
INFO:tensorflo

INFO:tensorflow:Saving checkpoints for 31096 into model_regular_10/model.ckpt.
INFO:tensorflow:Loss for final step: 1.1625944.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_regular_10/model.ckpt-31096
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_regular_10/model.ckpt-31096
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
z-score: [-1.8682247]
epoch 5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_regular_10/model.ckpt-31096
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_i

INFO:tensorflow:loss = 1.1483599, step = 37697 (56.407 sec)
INFO:tensorflow:global_step/sec: 1.4733
INFO:tensorflow:loss = 1.1830119, step = 37797 (67.875 sec)
INFO:tensorflow:global_step/sec: 1.69254
INFO:tensorflow:loss = 1.1380459, step = 37897 (59.083 sec)
INFO:tensorflow:global_step/sec: 1.7317
INFO:tensorflow:loss = 1.2326388, step = 37997 (57.747 sec)
INFO:tensorflow:global_step/sec: 1.74042
INFO:tensorflow:loss = 1.1828094, step = 38097 (57.460 sec)
INFO:tensorflow:global_step/sec: 1.56921
INFO:tensorflow:loss = 1.2007426, step = 38197 (63.724 sec)
INFO:tensorflow:global_step/sec: 1.39724
INFO:tensorflow:loss = 1.1429255, step = 38297 (71.570 sec)
INFO:tensorflow:global_step/sec: 1.4559
INFO:tensorflow:loss = 1.1788611, step = 38397 (68.686 sec)
INFO:tensorflow:Saving checkpoints for 38496 into model_regular_10/model.ckpt.
INFO:tensorflow:global_step/sec: 1.34127
INFO:tensorflow:loss = 1.2055646, step = 38497 (74.556 sec)
INFO:tensorflow:global_step/sec: 1.32675
INFO:tensorflow

INFO:tensorflow:global_step/sec: 1.36919
INFO:tensorflow:loss = 1.0916126, step = 44371 (73.038 sec)
INFO:tensorflow:global_step/sec: 1.36667
INFO:tensorflow:loss = 1.1920589, step = 44471 (73.170 sec)
INFO:tensorflow:global_step/sec: 1.43694
INFO:tensorflow:loss = 1.0848583, step = 44571 (69.591 sec)
INFO:tensorflow:Saving checkpoints for 44589 into model_regular_10/model.ckpt.
INFO:tensorflow:global_step/sec: 1.53971
INFO:tensorflow:loss = 1.1427215, step = 44671 (64.947 sec)
INFO:tensorflow:global_step/sec: 1.55118
INFO:tensorflow:loss = 1.192171, step = 44771 (64.467 sec)
INFO:tensorflow:global_step/sec: 1.50641
INFO:tensorflow:loss = 1.181183, step = 44871 (66.385 sec)
INFO:tensorflow:global_step/sec: 1.35572
INFO:tensorflow:loss = 1.2033372, step = 44971 (73.760 sec)
INFO:tensorflow:global_step/sec: 1.50518
INFO:tensorflow:loss = 1.1326003, step = 45071 (66.437 sec)
INFO:tensorflow:global_step/sec: 1.42387
INFO:tensorflow:loss = 1.1425176, step = 45171 (70.233 sec)
INFO:tensorflo

INFO:tensorflow:loss = 1.1573812, step = 49645 (57.149 sec)
INFO:tensorflow:global_step/sec: 1.74976
INFO:tensorflow:loss = 1.1267954, step = 49745 (57.151 sec)
INFO:tensorflow:global_step/sec: 1.74765
INFO:tensorflow:loss = 1.239674, step = 49845 (57.220 sec)
INFO:tensorflow:global_step/sec: 1.75277
INFO:tensorflow:loss = 1.2267162, step = 49945 (57.053 sec)
INFO:tensorflow:global_step/sec: 1.7531
INFO:tensorflow:loss = 1.1420275, step = 50045 (57.042 sec)
INFO:tensorflow:global_step/sec: 1.75391
INFO:tensorflow:loss = 1.1380354, step = 50145 (57.017 sec)
INFO:tensorflow:global_step/sec: 1.74026
INFO:tensorflow:loss = 1.203132, step = 50245 (57.461 sec)
INFO:tensorflow:global_step/sec: 1.75746
INFO:tensorflow:loss = 1.3099098, step = 50345 (56.900 sec)
INFO:tensorflow:global_step/sec: 1.71463
INFO:tensorflow:loss = 1.1133548, step = 50445 (58.322 sec)
INFO:tensorflow:global_step/sec: 1.75201
INFO:tensorflow:loss = 1.2730569, step = 50545 (57.077 sec)
INFO:tensorflow:global_step/sec: 1

In order to better visualize we now also plot the z-scores.

In [ ]:
# Plotting z-scores
x = range(1, epoch + 1)
plt.plot(x, Z_SCORE_LIST, label='z-score')
plt.xlabel('Epoch')
plt.ylabel('z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.savefig("z_score_{}_regular.png".format(dataset['secret_format']).replace(' ', ''))
plt.show()
plt.close()
print(Z_SCORE_LIST)

The distribution of the log-perplexities of potenital secrets is approximiately a normal distribution.
So, for visualisation we plot a normal distribution and show where the current secret is placed, given its probability. Here, the more left on the plot, the more likely the sequence is under the model.
The y-axis also captures the epochs of the training. The first epoch is at the bottom, the last at the top.

In [ ]:
x = np.linspace(-5, 5, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1), label='Normal distribution')
plt.plot(Z_SCORE_LIST, np.linspace(0, 0.35, len(Z_SCORE_LIST)), marker='x', color='red', label='Secret Probabilities')
plt.xlabel('Standard deviations from mean')
plt.ylabel('Probability and epoch of z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.show()

## Training a model with differentially private optimization
Now we will also train a model with differentially private optimization. This might take a while.

In [ ]:
dpsgd = True

In [ ]:
warm_start_from = {'warm_start_from': model_dir_dp} if load_model else {}

# Instantiate the tf.Estimator.
conf = tf.estimator.RunConfig(save_summary_steps=1000)
lm_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=model_dir_dp,
                                       config=conf,
                                       **warm_start_from)

In [ ]:
# Training loop.
steps_per_epoch = NB_TRAIN // batch_len
Z_SCORE_LIST = []
EPSILON_LIST = []
for epoch in range(1, epochs + 1):
    print('epoch', epoch)
    # Train the model for one epoch.
    lm_classifier.train(input_fn=train_input_fn, steps=steps_per_epoch)

    if epoch % 5 == 1:
        name_input_fn = [('Train', train_input_fn), ('Eval', eval_input_fn)]
        for name, input_fn in name_input_fn:
            # Evaluate the model and print results
            eval_results = lm_classifier.evaluate(input_fn=input_fn)
            result_tuple = (epoch, eval_results['accuracy'], eval_results['loss'])
            print(name, 'accuracy after %d epochs is: %.3f (%.4f)' % result_tuple)

    z_score = estimate_z_score(estimator=lm_classifier,
                                secret=secret_sequence,
                                secret_format=dataset['secret_format'],
                                dictionary=dataset['dictionary'],
                                seed=seed + 1,
                                sample_size=1000)
    Z_SCORE_LIST.append(z_score)
    print("z-score: {}".format(z_score))        

In [ ]:
# Plotting z-scores
x = range(1, epoch + 1)
plt.plot(x, Z_SCORE_LIST, label='z-score')
plt.xlabel('Epoch')
plt.ylabel('z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.savefig("z_score_{}_dp.png".format(dataset['secret_format']).replace(' ', ''))
plt.show()
plt.close()
print(Z_SCORE_LIST)

# If we are using DP Optimization, we want to plot the epsilons, too
if dpsgd:
    plt.plot(x, EPSILON_LIST, label='epsilon')
    plt.xlabel('Epoch')
    plt.ylabel('epsilon')
    plt.legend()
    plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
    plt.savefig("epsilon_{}_dp.png".format(dataset['secret_format']).replace(' ', ''))
    plt.show()
    plt.close()
    print(EPSILON_LIST)

In [ ]:
x = np.linspace(-5, 5, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1), label='Normal distribution')
plt.plot(Z_SCORE_LIST, np.linspace(0, 0.35, 10), marker='x', color='red', label='Secret Probabilities')
plt.xlabel('Standard deviations from mean')
plt.ylabel('Probability and epoch of z-score')
plt.legend()
plt.title('Secret: {}'.format(dataset['secret_plain'].replace(' ', '').replace('_', ' ')))
plt.show()
